LAS file Processing Tools

# Tools for Deriving Products from LAS files
## Introduction
This jupyter notebook is designed to enable the efficient processing of LAS files. LAS files are an industry standard binary format for storing aerial LiDAR data. The python script in this notebook completes a number of processes useful for analysis. 
**Note:  As this script uses the arcpy package, it can only be run on a  machine with Arcgis Pro installed although ArcGIS Pro doesn't need to be running.**
## Workflow
The following processes are carried out in this script.
1. A Las dataset is created from the LAS files. A LAS dataset is a file storage container for LAS files used by ESRI
2. The ground surface is extracted and classified. This is the process where all the points in the cloud are processed and those deemed to form the bare earth surface are given a classification of "2" which is the designation for ground points in the LAS specification.
3. A digital Surface Model (DSM) of the pointcloud is generated. This is a raster file with the pixels representing the elevation of the non classified cloud.
4. A Digital Terrain Model is created of the ground. This is a raster file with the pixels representing the elecation of the points designated as bare earth.
5. A difference raster is created by subtracting the DTM from the DSM to show the vegetation height in the area of interest
6. If a shapefile is included containing a line feature class representing cross sections then these will be extracted and plotted in the notebook as well as being exported as a jpeg and a CSV file.

** A sample dataset containing a LAS file and a shapefile can be downloaded [here](https://drive.google.com/drive/folders/1oWOcZGH8XyItQWyAD-vEmMEGzo5CKYn_?usp=sharing)



In [62]:
import arcpy
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import os

In [70]:
arcpy_env_workspace = "C:\\DSM_Tools\\"  # enter path and name of workspace here
pointcloud_path = r"C:\DSM_Tools\LAS_Processing_Tools\Pointcloud_Data\Pointcloud.las" #enter the path to your LAS files
las_dataset_path = "C:\\DSM_Tools\\LAS_Processing_Tools\\Pointcloud_Data\\"# enter las dataset path here
raster_folder = "C:\\DSM_Tools\\LAS_Processing_Tools\\rasters\\"  #enter path to where output rasters ar to be stored
ground_dsm = "grfhgd_DSM.tif"
raster_full_path = os.path.join(raster_folder, ground_dsm)

In [71]:
#create a temporary geodatabase in workspace
if arcpy.Exists(arcpy_env_workspace +"temp.gdb"):
    arcpy.Delete_management(arcpy_env_workspace +"temp.gdb")
arcpy.management.CreateFileGDB(arcpy.env.workspace, 'temp')

<Result 'C:\\DSM_Tools\\temp.gdb'>

In [73]:
#First make las dataset from pointcloud
arcpy.management.CreateLasDataset(pointcloud_path,
                                  las_dataset_path+"Pointcloud.lasd", "NO_RECURSION", None,
                                  "PROJCS['British_National_Grid',GEOGCS['GCS_OSGB_1936',"
                                  "DATUM['D_OSGB_1936',SPHEROID['Airy_1830',6377563.396,299.3249646]],"
                                  "PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],"
                                  "PROJECTION['Transverse_Mercator'],"
                                  "PARAMETER['False_Easting',400000.0],PARAMETER['False_Northing',-100000.0],"
                                  "PARAMETER['Central_Meridian',-2.0],PARAMETER['Scale_Factor',0.9996012717],"
                                  "PARAMETER['Latitude_Of_Origin',49.0],UNIT['Meter',1.0]]",
                                  "COMPUTE_STATS", "ABSOLUTE_PATHS", "NO_FILES")

<Result 'C:\\DSM_Tools\\LAS_Processing_Tools\\Pointcloud_Data\\Pointcloud.lasd'>

In [75]:
##Classify the ground surface

arcpy.ddd.ClassifyLasGround(las_dataset_path+"Pointcloud.lasd", 
                            "AGGRESSIVE", "RECLASSIFY_GROUND", None, "COMPUTE_STATS", "DEFAULT", None, 
                            "PROCESS_EXTENT")


<Result 'C:\\DSM_Tools\\LAS_Processing_Tools\\Pointcloud_Data\\Pointcloud.lasd'>

In [76]:
 
if arcpy.Exists("Pointcloud_LasDatasetLayer"):
    arcpy.Delete_management("Pointcloud_LasDatasetLayer")
    

arcpy.management.MakeLasDatasetLayer(r"C:\Tool\DSM_Processing\Pointcloud_Data\Pointcloud.lasd", "Pointcloud_LasDatasetLayer", "2", None, "INCLUDE_UNFLAGGED", "INCLUDE_SYNTHETIC", "INCLUDE_KEYPOINT", "EXCLUDE_WITHHELD", None, "INCLUDE_OVERLAP")

<Result 'Pointcloud_LasDatasetLayer'>

In [77]:
if arcpy.Exists(raster_full_path):
    arcpy.Delete_management(raster_full_path)
arcpy.conversion.LasDatasetToRaster("Pointcloud_LasDatasetLayer", raster_full_path, "ELEVATION", "BINNING AVERAGE LINEAR", "FLOAT", "CELLSIZE", 0.1, 1)


<Result 'C:\\DSM_Tools\\LAS_Processing_Tools\\rasters\\grfhgd_DSM.tif'>